In [ ]:
import sys
!{sys.executable} -m pip install pandas pybaseball

In [1]:
from pybaseball import playerid_lookup, team_batting, team_pitching, pitching_stats, playerid_lookup, statcast_batter, schedule_and_record

In [19]:
# team xFIP (last 3 seasons)
team_xfip_df = team_pitching(2021, 2023, ind=0)
team_xfip = team_xfip_df['xFIP'].loc[team_xfip_df['Team'] == 'TEX']
team_xfip

20    4.34
Name: xFIP, dtype: float64

In [28]:
# starting pitcher xFIP (last 3 seasons)
starter_xfip_df = pitching_stats(2021, 2023, ind=0)
starter_xfip = starter_xfip_df['xFIP'].loc[starter_xfip_df['Name'] == 'Nathan Eovaldi']
starter_xfip

43    3.63
Name: xFIP, dtype: float64

In [ ]:
===============================================================================================================================

In [1]:
import pandas as pd

In [ ]:
team_xwoba_df = pd.read_csv('data/team_xwoba.csv')
player_xwoba_df = pd.read_csv('data/player_xwoba.csv')

In [13]:
# training data - game results from 2017-2022
train_data = {
    'away_team': [],
    'away_score': [],
    'away_team_xwoba': [],
    'away_lineup_xwoba': [],
    'away_team_xfip': [],
    'away_starter_xfip': [],
    'away_close_win_pct': [],
    'away_run_diff': [],
    'home_team': [],
    'home_score': [],
    'home_team_xwoba': [],
    'home_lineup_xwoba': [],
    'home_team_xfip': [],
    'home_starter_xfip': [],
    'home_close_win_pct': [],
    'home_run_diff': []
}

In [15]:
# enter individual game data into the dictionary
def write_data(data, year):
    df = pd.read_csv(f'data/gamelogs/gl{year}.txt')

    for index, row in df.iterrows():
        data['away_team'] = row['away_team']
        data['away_score'] = row['away_score']
        data['away_team_xwoba'] = get_team_xwoba(year, row['away_team'])
        data['away_lineup_xwoba'] = get_team_xwoba(year, [row['away_1_name'], row['away_2_name'], row['away_3_name'],
                                                                row['away_4_name'], row['away_5_name'], row['away_6_name'],
                                                                row['away_7_name'], row['away_8_name'], row['away_9_name']])
        data['away_team_xfip'] = get_team_xfip(year, row['away_team'])
        data['away_starter_xfip'] = get_starter_xfip(year, row['away_starter'])
        data['away_close_win_pct'] = get_close_win_pct(year, row['away_team'])
        data['away_run_diff'] = get_run_diff(year, row['away_team'])
        data['home_team'] = row['home_team']
        data['home_score'] = row['home_score']
        data['home_team_xwoba'] = get_team_xwoba(year, row['home_team'])
        data['home_lineup_xwoba'] = get_team_xwoba(year, [row['home_1_name'], row['home_2_name'], row['home_3_name'],
                                                                row['home_4_name'], row['home_5_name'], row['home_6_name'],
                                                                row['home_7_name'], row['home_8_name'], row['home_9_name']])
        data['home_team_xfip'] = get_team_xfip(year, row['home_team'])
        data['home_starter_xfip'] = get_starter_xfip(year, row['home_starter'])
        data['home_close_win_pct'] = get_close_win_pct(year, row['home_team'])
        data['home_run_diff'] = get_run_diff(year, row['home_team'])

In [30]:
tc_retro = {
    'HOU': 'HOU',
    'TEX': 'TEX',
    'ANA': 'LAA',
    'OAK': 'OAK',
    'SEA': 'SEA',

    'KCA': 'KC',
    'DET': 'DET',
    'CLE': 'CLE',
    'MIN': 'MIN',
    'CHA': 'CWS',

    'NYA': 'NYY',
    'BOS': 'BOS',
    'TBA': 'TB',
    'TOR': 'TOR',
    'BAL': 'BAL',
    
    'LAN': 'LAD',
    'SDN': 'SD',
    'COL': 'COL',
    'ARI': 'AZ',
    'SFN': 'SF',
    
    'MIL': 'MIL',
    'PIT': 'PIT',
    'SLN': 'STL',
    'CIN': 'CIN',
    'CHN': 'CHC',

    'NYN': 'NYM',
    'ATL': 'ATL',
    'PHI': 'PHI',
    'WAS': 'WSH',
    'MIA': 'MIA',
}

In [31]:
### DON'T NEED TO RUN AGAIN!!!
### but double check the csv file with Baseball-Reference data

# run through gamelogs to calculate close_win_pct and run_diff

team_metrics = {
    'team': [],
    'year': [],
    'close_games': [],
    'close_win_pct': [],
    'run_diff': []
}

for year in range(2015, 2024):
    df = pd.read_csv(f'data/gamelogs/gl{year}.txt')

    for team in tc_retro:
        total_rs = 0
        total_ra = 0
        close_wins = 0
        close_losses = 0
        
        for index, row in df.iterrows():
            if row['away_team'] != team and row['home_team'] != team:
                continue

            if row['away_team'] == team:
                rs = row['away_score']
                ra = row['home_score']
            elif row['home_team'] == team:
                rs = row['home_score']
                ra = row['away_score']
                
            total_rs += rs
            total_ra += ra

            if abs(rs - ra) <= 1:
                if rs > ra:
                    close_wins += 1
                elif ra > rs:
                    close_losses += 1

        team_metrics['team'].append(tc_retro[team])
        team_metrics['year'].append(year)
        team_metrics['close_games'].append(close_wins + close_losses)
        team_metrics['close_win_pct'].append(close_wins / (close_wins + close_losses))
        team_metrics['run_diff'].append(total_rs - total_ra)

In [33]:
# save results from previous cell to csv file

team_metrics_df = pd.DataFrame(team_metrics)
team_metrics_df.to_csv('data/team_metrics.csv', index=False)